In [ ]:
import ee
import geemap
import numpy as np
import math
import time 
import traceback
import sys,os
# Check if you are in a Jupyter notebook and skip os.chdir() for Jupyter
if '__file__' in globals():
    script_dir = os.path.dirname(os.path.abspath(__file__))
    os.chdir(script_dir)

from tqdm import trange
from Func.Basic_tools import *
from Func.New_Correct import *
from Func.Correct_filter import *
from Func.S2_filter import *

geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

# Geometric Distortion Correction Functions

## `CalDitor` Function

**Purpose:** This function performs geometric distortion correction for satellite imagery, specifically for ascending and descending orbits.

**Parameters:**
- **s1_ascending:** Sentinel-1 image in ascending orbit.
- **s1_descending:** Sentinel-1 image in descending orbit.
- **Bands_:** List of bands to process, default is `['VV_gamma0_flatDB', 'VH_gamma0_flatDB']`.
- **Origin_Method:** Method for initial distortion calculation, default is `'RS'`.
- **Method:** Specifies the correction method, either `'1'` or `'2'`.
- **AOI_buffer:** Area of Interest buffer for processing.
- **Origin_scale:** Scale for the original image, default is `10`.
- **projScale:** Projection scale, default is `30`.

**Functionality:**
1. **Slope Correction:** Applies a slope correction to the images using `my_slope_correction`.
2. **Z-score Normalization:** Normalizes the images to match their median values, reducing the impact of different backscatter values.
3. **Geometric Distortion Detection:**
   - Generates auxiliary lines for both ascending and descending images.
   - Uses `Line_Correct` or `Line_Correct2` based on the specified `Method` to detect layover and shadow distortions.
4. **Output Preparation:** Renames the resulting distortion bands to avoid errors.

**Returns:**
- Distortion masks for layover and shadow for both orbits.
- Auxiliary line data for both orbits.
- Volume metric dictionary and synthesis data.

---

## `Combin_AB` Function

**Purpose:** Combines the distortion masks from ascending and descending orbits to create a comprehensive distortion map and correct the imagery accordingly.

**Parameters:**
- **LeftLayoverA, RightLayoverA, ShadowA:** Distortion masks for ascending orbit.
- **LeftLayoverD, RightLayoverD, ShadowD:** Distortion masks for descending orbit.
- **volumetric_dict:** Dictionary containing processed images and parameters.
- **AOI_buffer:** Area of Interest buffer for processing.
- **restrict:** Boolean flag to choose between strict or lenient correction mode.

**Functionality:**
1. **Mask Calculation:**
   - Calculates masks for each type of distortion (left layover, right layover, shadow) for both orbits.
   - Combines these masks to create a total distortion map.

2. **Distortion Correction:**
   - **Strict Mode:** If both orbits show distortion, the area is masked out.
   - **Lenient Mode:** Uses the opposite orbit's image to replace distorted areas, if possible.

3. **Image Synthesis:**
   - Generates mean, max, and min composite images from corrected ascending and descending data.

**Returns:**
- `s1_unit_mean_`: Mean composite image.
- `s1_unit_max_`: Maximum composite image.
- `s1_unit_min_`: Minimum composite image.
- `DistorDict`: A dictionary containing all distortion masks and the combined distortion mask.

---

**Notes:**
- The functions use Earth Engine (`ee`) for image processing, which implies they are designed to work within Google Earth Engine's environment.
- The `Method` parameter in `CalDitor` affects how geometric distortions are detected and corrected, with different algorithms potentially providing different results or performance.
- The `restrict` parameter in `Combin_AB` allows users to choose between a strict correction where areas with distortions in both orbits are excluded, or a lenient approach where data from the opposite orbit is used to fill in gaps.


In [ ]:
def CalDitor(s1_ascending, s1_descending, Bands_=['VV_gamma0_flatDB', 'VH_gamma0_flatDB'], Origin_Method='RS', Method='1', 
             AOI_buffer=None, Origin_scale=10, projScale=30):
    
    # Apply slope correction
    volumetric_dict = my_slope_correction(s1_ascending, s1_descending, AOI_buffer, DEMNASA, Model, Origin_scale, DistorMethed=Origin_Method)

    # Z-score normalization and histogram matching
    Ascending_Img = volumetric_dict['ASCENDING'].select(Bands_)
    Descending_Img = volumetric_dict['DESCENDING'].select(Bands_)
    
    A_median = Ascending_Img.reduceRegion(**{
        'reducer': ee.Reducer.median(),
        'geometry': AOI_buffer
    }).toArray()
    D_median = Descending_Img.reduceRegion(**{
        'reducer': ee.Reducer.median(),
        'geometry': AOI_buffer
    }).toArray()
    
    Ascending_Img = Ascending_Img.subtract(A_median.subtract(D_median).getInfo())
    volumetric_dict['ASCENDING'] = replaceBands(volumetric_dict['ASCENDING'], Ascending_Img)

    # Detect geometric distortions based on linear relationships
    Templist_A = AuxiliaryLine2Point(volumetric_dict['ASCENDING'], volumetric_dict['ASCENDING_parms']['s1_azimuth_across'],
                                     volumetric_dict['ASCENDING_parms']['coordinates_dict'],
                                     volumetric_dict['ASCENDING_parms']['Auxiliarylines'],
                                     projScale)

    Templist_D = AuxiliaryLine2Point(volumetric_dict['DESCENDING'], volumetric_dict['DESCENDING_parms']['s1_azimuth_across'],
                                     volumetric_dict['DESCENDING_parms']['coordinates_dict'],
                                     volumetric_dict['DESCENDING_parms']['Auxiliarylines'],
                                     projScale)

    if Method == '1':
        LeftLayoverA, RightLayoverA, ShadowA = Line_Correct(volumetric_dict['ASCENDING'], AOI_buffer, Templist_A, 'ASCENDING',
                                                           volumetric_dict['ASCENDING_parms']['proj'], projScale, Origin_scale,
                                                           filt_distance=False, save_peak=False, line_points_connect=True, Peak_Llay=True, Peak_shdow=True, Peak_Rlay=True)
        LeftLayoverD, RightLayoverD, ShadowD = Line_Correct(volumetric_dict['DESCENDING'], AOI_buffer, Templist_D, 'DESCENDING',
                                                           volumetric_dict['DESCENDING_parms']['proj'], projScale, Origin_scale,
                                                           filt_distance=False, save_peak=False, line_points_connect=True, Peak_Llay=True, Peak_shdow=True, Peak_Rlay=True)
    elif Method == '2':
        LeftLayoverA, RightLayoverA, ShadowA = Line_Correct2(volumetric_dict['ASCENDING'], AOI_buffer, Templist_A, 'ASCENDING',
                                                             volumetric_dict['ASCENDING_parms']['proj'], projScale, Origin_scale)
        LeftLayoverD, RightLayoverD, ShadowD = Line_Correct2(volumetric_dict['DESCENDING'], AOI_buffer, Templist_D, 'DESCENDING',
                                                             volumetric_dict['DESCENDING_parms']['proj'], projScale, Origin_scale)
    else:
        print(f'Method = {Method}')
        raise Exception('Method should be either 1 or 2')

    # Rename bands to avoid constant error
    LeftLayoverA = LeftLayoverA.rename('LeftLayoverA')
    RightLayoverA = RightLayoverA.rename('RightLayoverA')
    ShadowA = ShadowA.rename('ShadowA')
    LeftLayoverD = LeftLayoverD.rename('LeftLayoverD')
    RightLayoverD = RightLayoverD.rename('RightLayoverD')
    ShadowD = ShadowD.rename('ShadowD')

    return LeftLayoverA, RightLayoverA, ShadowA, Templist_A, LeftLayoverD, RightLayoverD, ShadowD, Templist_D, volumetric_dict, synthesis

def Combin_AB(LeftLayoverA, RightLayoverA, ShadowA, LeftLayoverD, RightLayoverD, ShadowD, 
              volumetric_dict, AOI_buffer=None, restrict=False):
    
    def Cal_mask(LeftLayover, RightLayover, Shadow, AOI_buffer):
        # Check if images are empty
        left_empty = LeftLayover.bandNames().length().eq(0)
        right_empty = RightLayover.bandNames().length().eq(0)
        shadow_empty = Shadow.bandNames().length().eq(0)

        # Combine non-empty images
        result = ee.Image(ee.Algorithms.If(left_empty, ee.Image(), LeftLayover))
        result = ee.Image(ee.Algorithms.If(right_empty, result, result.Or(RightLayover)))
        result = ee.Image(ee.Algorithms.If(shadow_empty, result, result.Or(Shadow)))
        return result.clip(AOI_buffer)

    LeftLayoverA_, RightLayoverA_, ShadowA_ = LeftLayoverA.mask().clip(AOI_buffer), RightLayoverA.mask().clip(AOI_buffer), ShadowA.mask().clip(AOI_buffer)
    LeftLayoverD_, RightLayoverD_, ShadowD_ = LeftLayoverD.mask().clip(AOI_buffer), RightLayoverD.mask().clip(AOI_buffer), ShadowD.mask().clip(AOI_buffer)

    A_mask_ = Cal_mask(LeftLayoverA_, RightLayoverA_, ShadowA_, AOI_buffer)
    D_mask_ = Cal_mask(LeftLayoverD_, RightLayoverD_, ShadowD_, AOI_buffer)
    All_distor = A_mask_.And(D_mask_)

    DistorDict = {
        'LeftLayoverA': LeftLayoverA, 'RightLayoverA': RightLayoverA, 'ShadowA': ShadowA, 'A_mask_': A_mask_,
        'LeftLayoverD': LeftLayoverD, 'RightLayoverD': RightLayoverD, 'ShadowD': ShadowD, 'D_mask_': D_mask_,
        'All_distor': All_distor
    }
    
    # Check if distortions exist
    A_empty = A_mask_.bandNames().contains('constant')
    A_empty = ee.Number(ee.Algorithms.If(A_empty, 1, 0))
    D_empty = D_mask_.bandNames().contains('constant')
    D_empty = ee.Number(ee.Algorithms.If(D_empty, 1, 0))

    s1_ascending_flat = volumetric_dict['ASCENDING'].select(["VV_gamma0_flatDB", "VH_gamma0_flatDB"])
    s1_descending_flat = volumetric_dict['DESCENDING'].select(["VV_gamma0_flatDB", "VH_gamma0_flatDB"])
    
    if restrict:
        # Strict mode: If both A and D have distortions, the area is set to empty
        s1_ascending_ = ee.Image(ee.Algorithms.If(A_empty, s1_ascending_flat, s1_ascending_flat.updateMask(A_mask_.Not())))
        s1_descending_ = ee.Image(ee.Algorithms.If(D_empty, s1_descending_flat, s1_descending_flat.updateMask(D_mask_.Not())))
        Combin_AD = ee.ImageCollection([s1_ascending_, s1_descending_])
        s1_unit_mean_ = Combin_AD.mean().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
        s1_unit_max_ = Combin_AD.max().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
        s1_unit_min_ = Combin_AD.min().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
    else:
        # Lenient mode: If there are distortions, prioritize using the opposite track image for replacement
        s1_ascending_ = ee.Image(ee.Algorithms.If(A_empty, ee.Image(), s1_ascending_flat.where(A_mask_, s1_descending_flat)))
        s1_descending_ = ee.Image(ee.Algorithms.If(D_empty, ee.Image(), s1_descending_flat.where(D_mask_, s1_ascending_flat)))
        Combin_AD = ee.ImageCollection([s1_ascending_, s1_descending_])
        s1_unit_mean_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty), 
                                                   s1_ascending_flat.add(s1_descending_flat).divide(2), 
                                                   Combin_AD.mean())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

        s1_unit_max_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                                                  s1_ascending_flat.max(s1_descending_flat),
                                                  Combin_AD.max())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

        s1_unit_min_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                                                  s1_ascending_flat.min(s1_descending_flat),
                                                  Combin_AD.min())).reproject(volumetric_dict['ASCENDING_parms']['proj'])
    
    return s1_unit_mean_, s1_unit_max_, s1_unit_min_, DistorDict


# Cluster Extraction Function

## `Cluster_math` Function

**Purpose:** This function performs various clustering algorithms on satellite imagery to extract features or segment the image based on different clustering methods.

**Parameters:**
- **method:** Clustering method to use. Options include:
  - `'Kmean'`: K-means clustering.
  - `'SNIC'`: Simple Non-Iterative Clustering.
  - `'SNIC_Kmean'`: SNIC followed by K-means.
  - `'LVQ'`: Learning Vector Quantization.
  - `'Xmeans'`: X-means clustering.
  - `'Cobweb'`: Cobweb clustering.
  - `'CascadeKMeans'`: Cascade K-means clustering.
- **img:** The input Earth Engine image to cluster.
- **bands:** List of bands from the image to use for clustering.
- **index:** A string identifier for saving results.
- **visual:** Boolean, if `True`, displays the clustering results on a map.
- **save:** Boolean, if `True`, saves the clustering results as a GeoTIFF file.
- **region:** Optional, the region to clip the image to before clustering.
- **proj:** Optional, the projection to reproject the image to before clustering.

**Functionality:**
1. **Image Preparation:**
   - Selects specified bands and clips the image to the region.
   - Normalizes the image using min-max normalization.

2. **Clustering:**
   - Applies the clustering method specified by `method`:
     - **Kmean:** Standard K-means clustering.
     - **SNIC:** Simple Non-Iterative Clustering, with optional reprojection.
     - **SNIC_Kmean:** SNIC followed by K-means, with options to include or exclude the original image in the second clustering step.
     - **LVQ, Xmeans, Cobweb, CascadeKMeans:** Other clustering algorithms from the `Cluster_extract` module.

3. **Visualization:**
   - If `visual` is `True`, creates a map centered on the region and adds layers for the original image and clustering results.

4. **Saving Results:**
   - If `save` is `True`, exports the clustering results as GeoTIFF files.

**Returns:**
- **Map:** A `geemap.Map` object if `visual` is `True`, otherwise `None`.
- **result:** The clustered image. For `'SNIC_Kmean'`, it returns an image with bands from both clustering steps.

**Notes:**
- The function uses the `Cluster_extract` module from `Func.Extract_algorithm` for clustering algorithms.
- `'SNIC_Kmean'` has an additional step where it performs K-means clustering twice, once without and once with the original image data.
- The `Geemap_export` function is assumed to be defined elsewhere for exporting images.
- The `minmax_norm` function is used for image normalization but not defined here.

---

**Usage Example:**
```python
# Example usage of the function
img = ee.Image('COPERNICUS/S2_SR/20210101T023659_20210101T024330_T52SDC')
bands = ['B4', 'B3', 'B2']
region = ee.Geometry.Point([107.76, 27.51]).buffer(10000)
result = Cluster_math('Kmean', img, bands, 'example', visual=True, save=True, region=region)


In [ ]:
from Func.Extract_algorithm import Cluster_extract as Cluster

def Cluster_math(method: str, img, bands: list, index: str, visual: bool, save: bool, region=None,proj=None):
    '''method ('Kmean','SNIC','SNIC_Kmean','LVQ','Xmeans','Cobweb','CascadeKMeans')'''
    img = img.select(bands).clip(region)
    img = minmax_norm(img,bands,region,scale=Origin_scale,withbound=False)

    if proj:
        img = img.reproject(proj)
    if method == 'Kmean':
        result = Cluster.afn_Kmeans(img, region)
    elif method == 'Cobweb':
        result = Cluster.afn_Cobweb(img, region)
    elif method == 'Xmeans':
        result = Cluster.afn_Xmeans(img, region)
    elif method == 'LVQ':
        result = Cluster.afn_LVQ(img, region)
    elif method == 'CascadeKMeans':
        result = Cluster.afn_CascadeKMeans(img, region)
    elif method == 'SNIC':
        result = Cluster.afn_SNIC(img)
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(proj)
    elif method == 'SNIC_Kmean':
        result = Cluster.afn_SNIC(img)
        # 默认舍弃cluster和seed
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(proj)
        result0 = Cluster.afn_Kmeans(result, region)  # 原始图像不参与
        result1 = Cluster.afn_Kmeans(result.addBands(img), region)  # 原始图像参与    .unmask(10)

    if visual:
        Map = geemap.Map(basemap='HYBRID')  #
        Map.centerObject(region, zoom=15)
        Map.addLayer(img.select(0), {'min': 0, 'max': 1}, 'Origin')
        if method in ['Kmean', 'Cobweb', 'Xmeans', 'LVQ', 'CascadeKMeans']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC':
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC_Kmean':
            Map.addLayer(result0.randomVisualizer(), {}, 'SNIC_Kmean_NoOrigin')
            Map.addLayer(result1.randomVisualizer(), {}, 'SNIC_Kmean_YesOrigin')
        else:
            print('Please check your method str')
    else:
        Map = None
    if save:
        if method == 'SNIC_Kmean':
            Geemap_export(filename=index + 'NoOrigin' + method + '.tif', collection=False, image=result0,
                          region=region, scale=10)
            Geemap_export(filename=index + 'YesOrigin' + method + '.tif', collection=False, image=result1,
                          region=region, scale=10)
        else:
            Geemap_export(filename=index + method + '.tif', collection=False, image=result, region=region, scale=10)
        pass
    if method == 'SNIC_Kmean':
        return Map, result0.addBands(result1)
    else:
        return Map, result

# Adaptive Thresholding Function

## `Bandmath` Function

**Purpose:** This function applies adaptive thresholding techniques to a single band of a satellite image to segment it based on different thresholding methods.

**Parameters:**
- **method:** Thresholding method to use. Options include:
  - `'otsu'`: Otsu's method for automatic threshold selection.
  - `'minimum'`: Minimum error thresholding.
  - `'yen'`: Yen's method for threshold selection.
  - `'isodata'`: Iterative self-organizing data analysis technique.
- **img:** The input Earth Engine image. Should be a single band image.
- **band:** The band name to process.
- **index:** A string identifier for saving results.
- **visual:** Boolean, if `True`, displays the thresholding results on a map.
- **save:** Boolean, if `True`, saves the thresholding results as a GeoTIFF file.
- **region:** Optional, the region to clip the image to before thresholding.
- **proj:** Optional, the projection to reproject the image to before thresholding.

**Functionality:**
1. **Image Preparation:**
   - Selects the specified band and clips the image to the region.
   - Normalizes the image using min-max normalization.
   - Reprojects the image to the specified projection if provided.

2. **Thresholding:**
   - Depending on the method:
     - **Otsu:** Calculates the threshold using Otsu's method on the image histogram.
     - **Minimum:** Uses the minimum error method to find the optimal threshold.
     - **Yen:** Applies Yen's thresholding algorithm.
     - **Isodata:** Uses the ISODATA algorithm for threshold selection.
   - Applies the threshold to create a binary image.

3. **Visualization:**
   - If `visual` is `True`, creates a map centered on the region, adds layers for the original image, and the thresholded result.

4. **Saving Results:**
   - If `save` is `True`, exports the thresholded image as a GeoTIFF file.

**Returns:**
- **Map:** A `geemap.Map` object if `visual` is `True`, otherwise `None`.
- **result:** The thresholded image.

**Notes:**
- The function uses the `Adaptive_threshold` module from `Func.Extract_algorithm` for thresholding algorithms.
- Functions like `get_histogram`, `GetHistAndBoundary`, `minmax_norm`, and `Geemap_export` are assumed to be defined elsewhere or imported from other modules.
- The image must be a single band for this function to work correctly.

---

**Usage Example:**
```python
# Example usage of the function
img = ee.Image('COPERNICUS/S2_SR/20210101T023659_20210101T024330_T52SDC')
region = ee.Geometry.Point([107.76, 27.51]).buffer(10000)
result = Bandmath('otsu', img, 'B4', 'example', visual=True, save=True, region=region)


In [ ]:
from Func.Extract_algorithm import Adaptive_threshold as Adap

def Bandmath(method:str,img,band,index:str,visual:bool,save:bool,region=None,proj=None):
    '''
    method = ['otsu','minimum','yen','isodata']
    img: 仅单波段图像
    '''
    img = img.select(band).clip(region)
    img = minmax_norm(img,band,region,scale=Origin_scale,withbound=False)
    img = img.reproject(proj).clip(region)

    if method == 'otsu':
        histogram = get_histogram(img,region=region,scale=Origin_scale)
        Threshould_value = Adap.afn_otsu(histogram)
        result = img.select(0).gt(Threshould_value).clip(region)  #
        print('Threshould value is {}'.format(Threshould_value.getInfo()))

    elif method == 'minimum':
        bin_centers, counts,_  = GetHistAndBoundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_minimum(bin_centers, counts,max_num_iter = 10000)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))
        
    elif method == 'yen':
        bin_centers, counts,_ = GetHistAndBoundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_yen(bin_centers, counts)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))
    
    elif method == 'isodata':
        bin_centers, counts, bucketWidth = GetHistAndBoundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_isodata(bin_centers, counts,bucketWidth)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))

    if visual:
        Map = geemap.Map(basemap='HYBRID') 
        Map.centerObject(region, zoom=15)
        Map.addLayer(img.select(0), {'min': 0, 'max': 1}, 'Origin')
        if method in ['otsu','minimum','yen','isodata']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        else:
            print('Wrong visual! Please check your method str')
    else:
        Map = None

    if save:
        if method in ['otsu','minimum','yen','isodata']:
            Geemap_export(filename=index+method+'.tif',collection=False,
                          image=result,region=region,scale=10)
        else:
            print('Wrong save! Please check your method str')

    return Map,result

# Glacial Lake Data Processing

In [ ]:
# Preloading Glacial Lake Data
Glacial_lake = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Glacial_lake/Southwest_WangAndChen_mergeGL')

# Calculate geometry, centroid, and minimum bounding rectangle
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_C = Glacial_lake.map(Geo_ext).map(Centrid_set)  # Add properties, modify geometry, calculate centroid
Glacial_lake_R = Glacial_lake.map(Rectangle_set)             # Calculate minimum bounding rectangle

# Extract properties as lists
Num_list = Glacial_lake.size().getInfo()
Glacial_lake_A_GeoList = Glacial_lake.toList(Num_list)
Glacial_lake_C_CentriodList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(), ['Centroid']).get('list'))
Glacial_lake_R_RectangleList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(), ['Rectangle']).get('list'))

# Determine time range
year = '2019'
START_DATE = ee.Date(year + '-06-01')
END_DATE = ee.Date(year + '-08-30')
S2_START_DATE = ee.Date(year + '-06-01')
S2_END_DATE = ee.Date(year + '-09-30')
TIME_LEN = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(), 'days')

# DEM Selection
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)

# Geometric Distortion Parameters
models = ['volume', 'surface', None]  # Terrain correction models
Model = models[0]
Origin_scale = 10
projScale = 30

# S2 Cloudless Parameters
CLOUD_FILTER = 60         # Filter S2 images with cloud cover greater than the specified percentage
CLD_PRB_THRESH = 15       # Threshold for s2cloudless probability [0-100], originally set at 50
NIR_DRK_THRESH = 0.15     # Threshold for dark NIR pixels
CLD_PRJ_DIST = 1          # Distance to project cloud shadows from clouds
BUFFER = 50               # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER

# Classification Methods and Parameters
from Func.Extract_algorithm import Reprocess, save_parms
def make_dir(path):
    return path if os.path.exists(path) else os.makedirs(path) or (print(f"{path} created successfully"), path)[1]

save_dir = make_dir(f'D:\\Dataset_and_Demo\\temp\\{year} supplementary results')
os.chdir(save_dir)
Methods = ('Kmean', 'SNIC_Kmean', 'Xmeans', 'CascadeKMeans', 'otsu', 'isodata')
resultbands = (0, 1)
Imgs = ('s1_unit_mean_',)  # 's1_unit_max_', 's1_unit_min_'
Bands = (['VV_gamma0_flatDB'], ['VV_gamma0_flatDB', 'VH_gamma0_flatDB'])
mode = 'gpd'
savenmae = '{}SouthwestGLs'.format(year)


In [ ]:
for i in trange(0, Num_list):
    IoU_All = []
    Wrong_dataIndex = []
    try:
        start_time = time.time()
        AOI_Centriod = ee.Feature.geometry(ee.Feature(Glacial_lake_C_CentriodList.get(i)))
        AOI_GLFeature = ee.Feature(Glacial_lake_A_GeoList.get(i))
        AOI_GL = AOI_GLFeature.geometry()
        AOI_Rectangle = ee.Feature.geometry(ee.Feature(Glacial_lake_R_RectangleList.get(i)))

        # Calculate area
        # AOI_area = AOI_GL.area().divide(ee.Number(1000*1000)).getInfo()
        # AOI_perometer = AOI_GL.perimeter().divide(1000).getInfo()
        # circularity = AOI_area*3.1415926*4/AOI_perometer**2
        AOI_area = AOI_GLFeature.get('GL_Area_km').getInfo()
        circularity = AOI_GLFeature.get('GL_Circula').getInfo()

        # Shrink the glacial lake area
        AOI_ShrinKbuffer = AOI_GL.buffer(distance=math.log(AOI_area+1,10) * -3000 * circularity)
        AOI_ShrinKbuffer_Centroid = AOI_ShrinKbuffer.centroid()

        # Check if the shrunk area is empty, otherwise replace with AOI_area
        if AOI_ShrinKbuffer.coordinates().getInfo() != []:
            FilterBound = AOI_ShrinKbuffer_Centroid
        else:
            FilterBound = AOI_Centriod

        # The boundary should not be too large, otherwise it will affect the local adaptive extraction effect
        AOI_buffer = AOI_Rectangle.buffer(distance=math.log(AOI_area+1,5) * 1200 + 100).bounds()

        # Load images, apply filtering functions, filter by date, AOI_buffer is only used to check for null values
        s1_ascending, s1_descending = load_image_collection(AOI_buffer, START_DATE, END_DATE, MIDDLE_DATE, Filter='RefinedLee', FilterSize=30)

        # Use S2cloudless to produce a cloud-free composite image
        s2_sr_median = merge_s2_collection(AOI_buffer, START_DATE, END_DATE, CLOUD_FILTER, BUFFER, CLD_PRJ_DIST, CLD_PRB_THRESH, NIR_DRK_THRESH)
        
        # Check if the image is synthesized
        synthesis_a = ee.Image(s1_ascending).get('synthesis').getInfo()
        synthesis_d = ee.Image(s1_descending).get('synthesis').getInfo()
        if synthesis_a or synthesis_d:
            synthesis = 1
        else:
            synthesis = 0

        if synthesis:
            print('Image with synthesis')
            s1_ascending = s1_ascending.rename(['VV_gamma0_flatDB', 'VH_gamma0_flatDB', 'incAngle'])
            s1_descending = s1_descending.rename(['VV_gamma0_flatDB', 'VH_gamma0_flatDB', 'incAngle'])
            default_prj = s1_ascending.select(0).projection()
            Combin_ad = ee.ImageCollection([s1_ascending, s1_descending])
            s1_unit_mean_ = Combin_ad.mean().reproject(default_prj).clip(AOI_buffer)
            s1_unit_max_ = Combin_ad.max().reproject(default_prj).clip(AOI_buffer)
            s1_unit_min_ = Combin_ad.min().reproject(default_prj).clip(AOI_buffer)
            volumetric_dict = {}
            volumetric_dict['ASCENDING'] = s1_ascending.clip(AOI_buffer)
            volumetric_dict['ASCENDING_parms'] = {'proj': default_prj}
            volumetric_dict['DESCENDING'] = s1_descending.clip(AOI_buffer)
            volumetric_dict['DESCENDING_parms'] = {'proj': s1_descending.select(0).projection()}
        else:
            # Geometric distortion detection and merging of ascending and descending images
            LeftLayoverA, RightLayoverA, ShadowA, Templist_A, LeftLayoverD, RightLayoverD, ShadowD, Templist_D, volumetric_dict, synthesis = \
                CalDitor(s1_ascending, s1_descending, Bands_=['VV_gamma0_flatDB', 'VH_gamma0_flatDB'], Origin_Method='RS', Method='1',
                        AOI_buffer=AOI_buffer, Origin_scale=Origin_scale, projScale=projScale)
            
            s1_unit_mean_, s1_unit_max_, s1_unit_min_, DistorDict = \
                Combin_AB(LeftLayoverA, RightLayoverA, ShadowA, LeftLayoverD, RightLayoverD, ShadowD, volumetric_dict, AOI_buffer=AOI_buffer)
       
        if synthesis_a == 0:
            a_name, a_date, a_nodata = ee.List([s1_ascending.get('system:index'), s1_ascending.date().format('YYYY-MM-dd'), s1_ascending.get('numNodata')]).getInfo()
        else:
            a_name = 'None'; a_date = 'None'; a_nodata = 'All_have'

        if synthesis_d == 0:
            d_name, d_date, d_nodata = ee.List([s1_descending.get('system:index'), s1_descending.date().format('YYYY-MM-dd'), s1_descending.get('numNodata')]).getInfo()
        else:
            d_name = 'None'; d_date = 'None'; d_nodata = 'All_have'

        pd_dict = {'a_name': a_name, 'd_name': d_name, 'a_date': a_date,
                   'd_date': d_date, 'a_nodata': a_nodata, 'd_nodata': d_nodata}

        K = [1 if Method == 'SNIC_Kmean' else 0 for Method in Methods]

        for k, Method in zip(K, Methods):
            for img in Imgs:
                for Band in Bands:
                    Bands = (['VV_gamma0_flatDB'], ['VV_gamma0_flatDB', 'VH_gamma0_flatDB'])

                    if Method in ['Kmean', 'SNIC_Kmean', 'LVQ', 'Xmeans', 'CascadeKMeans']:
                        if len(Band) == 2:
                            Map, result = Cluster_math(method=Method, img=eval(img), bands=Band, index='',
                                                        visual=False, save=False, region=AOI_buffer,
                                                        proj=volumetric_dict['ASCENDING_parms']['proj'])
                        else:
                            print('Method 1 {} Band={}, continue'.format(Method, Band))
                            continue

                    elif Method in ['otsu', 'minimum', 'yen', 'isodata']:
                        if len(Band) == 1:
                            Map, result = Bandmath(method=Method, img=eval(img), band=Band, index='',
                                    visual=False, save=False, region=AOI_buffer,
                                    proj=volumetric_dict['ASCENDING_parms']['proj'])
                        else:
                            print('Method 2 {} Band={}, continue'.format(Method, Band))
                            continue
                    else:
                        print('Wrong method input!')

                    for resultband in range(k + 1):

                        # Convert classification map to vector, FilterBound can choose AOI_point or FilterBuffer
                        Union_ex = Reprocess.image2vector(result, resultband=resultband, GLarea=AOI_area, FilterBound=FilterBound, del_maxcount=False)
                        logname = savenmae + f'{i // 100 * 100}_{i // 100 * 100 + 100}.csv'
                        shpname = savenmae + f'{i // 100 * 100}_{i // 100 * 100 + 100}.shp'
                        # Export csv and shp
                        Result_dict = save_parms.write_pd(Union_ex, i, AOI_GL, img, mode=mode, Method=Method, Band=Band, WithOrigin=resultband, pd_dict=pd_dict,
                                              Area_real=AOI_area, logname=logname, shapname=shpname, calIoU=True, cal_resultArea=False, returnParms=True)
                        IoU_All.append(Result_dict['IoU'])
                        print('Write csv and shp, Method={} Img={} Band={}'.format(Method, img, Band))

        # When IoU is relatively small, IoU<0.6, download the images
        if np.sum(np.array(IoU_All) > 0.6) == 0:
                Geemap_export(fileDirname=f'{i:04d}' + '_' + 'Ascending_' + a_name + '.tif', collection=False, image=volumetric_dict['ASCENDING'], region=AOI_buffer, scale=10)
                Geemap_export(fileDirname=f'{i:04d}' + '_' + 'Descending_' + d_name + '.tif', collection=False, image=volumetric_dict['DESCENDING'], region=AOI_buffer, scale=10)
                Geemap_export(fileDirname=f'{i:04d}' + '_' + 's2a_sr_median' + '.tif', collection=False, image=s2_sr_median, region=AOI_buffer, scale=10)   
            
                # Export the merged result images
                Geemap_export(fileDirname=f'{i:04d}' + '_' + 's1_unit_mean_' + '.tif', collection=False, image=s1_unit_mean_, region=AOI_buffer, scale=10)
                Geemap_export(fileDirname=f'{i:04d}' + '_' + 's1_unit_max_' + '.tif', collection=False, image=s1_unit_max_, region=AOI_buffer, scale=10)
                Geemap_export(fileDirname=f'{i:04d}' + '_' + 's1_unit_min_' + '.tif', collection=False, image=s1_unit_min_, region=AOI_buffer, scale=10)
                
                if synthesis != 1:
                    # Export geometric distortion images
                    DistorA = LeftLayoverA.rename('LeftLayoverA').addBands(RightLayoverA.rename('RightLayoverA')).addBands(ShadowA.rename('ShadowA'))
                    if DistorA.bandNames().length().getInfo() != 0:
                        Geemap_export(fileDirname=f'{i:04d}' + '_' + 'DistortionA' + a_name + '.tif', image=DistorA, region=AOI_buffer, scale=10)

                    DistorD = LeftLayoverD.rename('LeftLayoverD').addBands(RightLayoverD.rename('RightLayoverD')).addBands(ShadowD.rename('ShadowD'))
                    if DistorD.bandNames().length().getInfo() != 0:
                        Geemap_export(fileDirname=f'{i:04d}' + '_' + 'DistortionD' + d_name + '.tif', image=DistorD, region=AOI_buffer, scale=10) 

        end_time = time.time()
        print('This function ran for %.5f seconds' % (end_time - start_time))
    except:
        # Record error information
        Wrong_dataIndex.append(i)
        with open('log.txt', 'a') as f:
            f.write('Wrong index = {}\n'.format(i))
            f.write(traceback.format_exc())
            f.write('\n')
        print('Error has been logged to log.txt')
